# Forex Pattern Mining - Algorithmic Framework

**Based on Research Paper:** "An Algorithmic Framework for Frequent Intraday Pattern Recognition and Exploitation in Forex Market"

This notebook implements a comprehensive pattern mining framework for discovering frequent patterns in forex price movements using statistical analysis and machine learning techniques.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully")
print("Forex Pattern Mining Framework Initialized")

In [ ]:
# Pattern Mining Framework Class
class ForexPatternMiner:
    def __init__(self, params):
        self.params = params
        self.patterns = []
        self.statistics = {}
        self.data = None
        
    def fetch_forex_data(self):
        """Generate realistic forex-like data for pattern analysis"""
        print(f"Generating {self.params['dataPoints']} data points for {self.params['currencyPair']['symbol']}...")
        
        n_points = self.params['dataPoints']
        
        # Base prices for different currency pairs
        base_prices = {
            'EURUSD': 1.2000, 'GBPUSD': 1.3500, 'USDJPY': 110.00,
            'USDCHF': 0.9200, 'AUDUSD': 0.7500, 'USDCAD': 1.2500
        }
        
        symbol = self.params['currencyPair']['symbol']
        base_price = base_prices.get(symbol, 1.0000)
        
        # Generate realistic price movements with patterns
        prices = []
        current_price = base_price
        
        for i in range(n_points):
            # Base random walk
            random_change = np.random.normal(0, 0.0008)  # Realistic forex volatility
            
            # Add trend components
            trend = np.sin(i / 500) * 0.0002  # Long-term trend
            cycle = np.sin(i / 50) * 0.0001   # Medium-term cycle
            
            # Inject patterns occasionally
            if i > 50 and np.random.random() > 0.98:  # 2% chance of pattern
                pattern_type = np.random.choice(['bullish', 'bearish', 'consolidation'])
                pattern_strength = np.random.uniform(0.0005, 0.002)
                
                if pattern_type == 'bullish':
                    random_change += pattern_strength
                elif pattern_type == 'bearish':
                    random_change -= pattern_strength
                # consolidation adds no directional bias
            
            # Apply all changes
            price_change = random_change + trend + cycle
            current_price *= (1 + price_change)
            
            prices.append(current_price)
        
        # Create OHLC data
        data = []
        for i, close_price in enumerate(prices):
            open_price = prices[i-1] if i > 0 else close_price
            
            # Generate realistic OHLC from close price
            spread = abs(np.random.normal(0, 0.0002))
            high = max(open_price, close_price) + spread * np.random.uniform(0, 1)
            low = min(open_price, close_price) - spread * np.random.uniform(0, 1)
            
            data.append({
                'timestamp': f'2024-01-01 {(i*15)//60:02d}:{(i*15)%60:02d}:00',
                'open': open_price,
                'high': high,
                'low': low,
                'close': close_price,
                'volume': np.random.randint(1000, 10000)
            })
        
        self.data = pd.DataFrame(data)
        print(f"Generated {len(self.data)} OHLC data points")
        return self.data
    
    def extract_patterns(self, data):
        """Extract frequent patterns using sliding window approach"""
        print("Extracting patterns using sliding window approach...")
        
        window_size = self.params['windowSize']
        min_support = self.params['minSupport']
        patterns_found = []
        
        # Calculate technical indicators first
        data = self.add_technical_indicators(data)
        
        # Sliding window pattern extraction
        for i in range(len(data) - window_size):
            window = data.iloc[i:i+window_size]
            
            # Extract pattern features
            pattern_features = self.extract_pattern_features(window)
            
            if pattern_features['pattern_strength'] > 0.1:  # Filter noise
                patterns_found.append({
                    'start_idx': i,
                    'end_idx': i + window_size,
                    'features': pattern_features,
                    'normalized_prices': pattern_features['normalized_prices'],
                    'pattern_type': pattern_features['pattern_type']
                })
        
        print(f"Found {len(patterns_found)} raw patterns")
        
        # Cluster similar patterns and find frequent ones
        frequent_patterns = self.find_frequent_patterns(patterns_found, min_support)
        
        print(f"Identified {len(frequent_patterns)} frequent patterns")
        return frequent_patterns
    
    def add_technical_indicators(self, data):
        """Add technical indicators to the data"""
        if not self.params.get('includeTechnicalIndicators', True):
            return data
        
        # RSI calculation
        rsi_period = self.params.get('rsiPeriod', 14)
        delta = data['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=rsi_period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_period).mean()
        rs = gain / loss
        data['rsi'] = 100 - (100 / (1 + rs))
        
        # Moving averages
        data['sma_20'] = data['close'].rolling(window=20).mean()
        data['ema_20'] = data['close'].ewm(span=20).mean()
        
        # Bollinger Bands
        bb_period = self.params.get('bollBandPeriod', 20)
        sma = data['close'].rolling(window=bb_period).mean()
        std = data['close'].rolling(window=bb_period).std()
        data['bb_upper'] = sma + (2 * std)
        data['bb_lower'] = sma - (2 * std)
        data['bb_position'] = (data['close'] - data['bb_lower']) / (data['bb_upper'] - data['bb_lower'])
        
        return data
    
    def extract_pattern_features(self, window):
        """Extract features from a price window"""
        prices = window['close'].values
        
        # Normalize prices
        normalized = (prices - prices[0]) / prices[0]
        
        # Calculate pattern features
        trend = normalized[-1] - normalized[0]  # Overall trend
        volatility = np.std(normalized)  # Volatility
        momentum = np.mean(np.diff(normalized))  # Average momentum
        pattern_strength = abs(trend) + volatility  # Combined strength
        
        # Price action features
        max_price = np.max(normalized)
        min_price = np.min(normalized)
        price_range = max_price - min_price
        
        # Directional features
        up_moves = np.sum(np.diff(normalized) > 0)
        down_moves = np.sum(np.diff(normalized) < 0)
        directional_bias = (up_moves - down_moves) / len(normalized)
        
        # Technical indicator features (if available)
        rsi_start = window['rsi'].iloc[0] if 'rsi' in window.columns else 50
        rsi_end = window['rsi'].iloc[-1] if 'rsi' in window.columns else 50
        rsi_change = rsi_end - rsi_start
        
        # Pattern classification
        pattern_type = self.classify_pattern(trend, volatility, momentum, directional_bias)
        
        return {
            'normalized_prices': normalized.tolist(),
            'trend': trend,
            'volatility': volatility,
            'momentum': momentum,
            'pattern_strength': pattern_strength,
            'price_range': price_range,
            'directional_bias': directional_bias,
            'rsi_change': rsi_change,
            'pattern_type': pattern_type
        }
    
    def classify_pattern(self, trend, volatility, momentum, directional_bias):
        """Classify pattern based on features"""
        # Noise filter
        noise_threshold = self.params.get('noiseFilter', 0.1) / 100
        
        if abs(trend) < noise_threshold and volatility < noise_threshold:
            return 'noise'
        
        # Strong directional patterns
        if trend > 0.003 and momentum > 0 and directional_bias > 0.2:
            return 'bullish'
        elif trend < -0.003 and momentum < 0 and directional_bias < -0.2:
            return 'bearish'
        
        # Consolidation patterns
        elif abs(trend) < 0.001 and volatility > 0.002:
            return 'neutral'
        
        # Default classification based on trend
        elif trend > 0.001:
            return 'bullish'
        elif trend < -0.001:
            return 'bearish'
        else:
            return 'neutral'
    
    def find_frequent_patterns(self, patterns, min_support):
        """Find frequent patterns using clustering and support threshold"""
        if len(patterns) < 10:
            return []
        
        # Extract feature vectors for clustering
        feature_vectors = []
        for pattern in patterns:
            features = pattern['features']
            vector = [
                features['trend'],
                features['volatility'],
                features['momentum'],
                features['pattern_strength'],
                features['directional_bias'],
                features['price_range']
            ]
            feature_vectors.append(vector)
        
        # Normalize features
        scaler = MinMaxScaler()
        normalized_features = scaler.fit_transform(feature_vectors)
        
        # Determine optimal number of clusters
        n_clusters = min(max(3, len(patterns) // 50), 20)  # Reasonable range
        
        # Perform clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(normalized_features)
        
        # Find frequent patterns (clusters with enough support)
        frequent_patterns = []
        total_patterns = len(patterns)
        
        for cluster_id in range(n_clusters):
            cluster_patterns = [patterns[i] for i in range(len(patterns)) if cluster_labels[i] == cluster_id]
            support = len(cluster_patterns) / total_patterns
            
            if support >= min_support and len(cluster_patterns) >= 3:
                # Create representative pattern for cluster
                representative = self.create_representative_pattern(cluster_patterns, cluster_id, support)
                frequent_patterns.append(representative)
        
        return frequent_patterns
    
    def create_representative_pattern(self, cluster_patterns, cluster_id, support):
        """Create a representative pattern from cluster"""
        # Calculate average features
        avg_features = {
            'trend': np.mean([p['features']['trend'] for p in cluster_patterns]),
            'volatility': np.mean([p['features']['volatility'] for p in cluster_patterns]),
            'momentum': np.mean([p['features']['momentum'] for p in cluster_patterns]),
            'pattern_strength': np.mean([p['features']['pattern_strength'] for p in cluster_patterns])
        }
        
        # Determine pattern type (most common in cluster)
        pattern_types = [p['pattern_type'] for p in cluster_patterns]
        pattern_type = max(set(pattern_types), key=pattern_types.count)
        
        # Calculate confidence based on pattern consistency
        confidence = self.calculate_confidence(cluster_patterns)
        
        # Calculate significance using statistical test
        significance = self.calculate_significance(cluster_patterns)
        
        # Estimate profitability (simplified)
        profitability = self.estimate_profitability(cluster_patterns, avg_features)
        
        # Get average pattern shape
        avg_shape = self.calculate_average_shape(cluster_patterns)
        
        return {
            'id': f'pattern_{cluster_id:03d}',
            'type': pattern_type,
            'support': support,
            'confidence': confidence,
            'significance': significance,
            'profitability': profitability,
            'winRate': max(0.3, confidence * 0.8),  # Estimated win rate
            'avgReturn': profitability,
            'maxDrawdown': abs(avg_features['trend']) * 0.5,  # Estimated
            'sharpeRatio': max(0, profitability / (avg_features['volatility'] + 0.001)),
            'pricePoints': avg_shape,
            'duration': len(avg_shape),
            'frequency': len(cluster_patterns),
            'occurrences': self.create_occurrences(cluster_patterns[:5])  # Sample occurrences
        }
    
    def calculate_confidence(self, cluster_patterns):
        """Calculate pattern confidence based on consistency"""
        if len(cluster_patterns) < 2:
            return 0.5
        
        # Measure consistency of pattern features
        trends = [p['features']['trend'] for p in cluster_patterns]
        volatilities = [p['features']['volatility'] for p in cluster_patterns]
        
        # Lower coefficient of variation = higher confidence
        trend_cv = np.std(trends) / (abs(np.mean(trends)) + 0.001)
        vol_cv = np.std(volatilities) / (np.mean(volatilities) + 0.001)
        
        # Confidence inversely related to variation
        base_confidence = 1.0 / (1.0 + trend_cv + vol_cv)
        
        # Boost confidence for larger clusters (more support)
        size_boost = min(0.2, len(cluster_patterns) / 100)
        
        return min(0.95, base_confidence + size_boost)
    
    def calculate_significance(self, cluster_patterns):
        """Calculate statistical significance using bootstrap method"""
        n_bootstrap = min(1000, self.params.get('bootstrapSamples', 1000))
        trends = [p['features']['trend'] for p in cluster_patterns]
        
        if len(trends) < 5:
            return 0.5
        
        # Bootstrap confidence interval
        bootstrap_means = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(trends, size=len(trends), replace=True)
            bootstrap_means.append(np.mean(sample))
        
        # Calculate significance (1 - p_value approximation)
        observed_mean = np.mean(trends)
        bootstrap_means = np.array(bootstrap_means)
        
        if abs(observed_mean) < 0.0001:  # Near-zero mean
            return 0.1
        
        # Approximate p-value
        p_value = np.mean(np.abs(bootstrap_means) <= np.abs(observed_mean))
        significance = max(0.1, 1 - p_value)
        
        return min(0.99, significance)
    
    def estimate_profitability(self, cluster_patterns, avg_features):
        """Estimate pattern profitability (simplified model)"""
        # Base profitability on trend direction and strength
        trend = avg_features['trend']
        volatility = avg_features['volatility']
        
        # Simple profitability model
        base_profit = trend * 100  # Convert to percentage
        
        # Adjust for risk (volatility)
        risk_adjusted = base_profit - (volatility * 50)  # Volatility penalty
        
        # Add some noise to make it realistic
        noise = np.random.normal(0, 0.02)  # ±2% noise
        
        profitability = risk_adjusted + noise
        
        # Clamp to reasonable range
        return max(-0.1, min(0.15, profitability))  # -10% to +15%
    
    def calculate_average_shape(self, cluster_patterns):
        """Calculate average pattern shape"""
        if not cluster_patterns:
            return []
        
        # Get all normalized price arrays
        shapes = [p['features']['normalized_prices'] for p in cluster_patterns]
        
        # Ensure all shapes have same length (pad if necessary)
        max_len = max(len(shape) for shape in shapes)
        padded_shapes = []
        
        for shape in shapes:
            if len(shape) < max_len:
                # Linear interpolation to extend
                padded = np.interp(np.linspace(0, len(shape)-1, max_len), 
                                 np.arange(len(shape)), shape)
                padded_shapes.append(padded)
            else:
                padded_shapes.append(shape[:max_len])
        
        # Calculate average
        avg_shape = np.mean(padded_shapes, axis=0)
        return avg_shape.tolist()
    
    def create_occurrences(self, sample_patterns):
        """Create sample pattern occurrences"""
        occurrences = []
        
        for i, pattern in enumerate(sample_patterns):
            # Simulate outcome based on trend
            trend = pattern['features']['trend']
            if trend > 0.002:
                outcome = 'profit' if np.random.random() > 0.3 else 'loss'
            elif trend < -0.002:
                outcome = 'profit' if np.random.random() > 0.4 else 'loss'
            else:
                outcome = np.random.choice(['profit', 'loss', 'breakeven'], p=[0.35, 0.35, 0.3])
            
            return_pct = trend * 100 + np.random.normal(0, 2)  # Add noise
            
            occurrences.append({
                'timestamp': f'2024-01-{i+1:02d} 10:00:00',
                'startIndex': pattern['start_idx'],
                'endIndex': pattern['end_idx'],
                'outcome': outcome,
                'returnPercentage': return_pct,
                'confidence': np.random.uniform(0.6, 0.9)
            })
        
        return occurrences
    
    def calculate_statistics(self, patterns):
        """Calculate comprehensive statistics"""
        if not patterns:
            return self._empty_statistics()
        
        # Basic statistics
        stats = {
            'totalPatterns': len(patterns),
            'uniquePatterns': len(set(p['type'] for p in patterns)),
            'avgConfidence': np.mean([p['confidence'] for p in patterns]),
            'avgSupport': np.mean([p['support'] for p in patterns]),
            'avgSignificance': np.mean([p['significance'] for p in patterns]),
        }
        
        # Performance statistics
        profitabilities = [p['profitability'] for p in patterns if p['profitability'] is not None]
        win_rates = [p['winRate'] for p in patterns if p['winRate'] is not None]
        sharpe_ratios = [p['sharpeRatio'] for p in patterns if p['sharpeRatio'] is not None]
        
        stats.update({
            'overallProfitability': np.mean(profitabilities) if profitabilities else 0,
            'avgWinRate': np.mean(win_rates) if win_rates else 0,
            'avgSharpeRatio': np.mean(sharpe_ratios) if sharpe_ratios else 0,
        })
        
        # Best/worst patterns
        if profitabilities:
            best_idx = np.argmax([p['profitability'] for p in patterns])
            worst_idx = np.argmin([p['profitability'] for p in patterns])
            stats['bestPattern'] = patterns[best_idx]['id']
            stats['worstPattern'] = patterns[worst_idx]['id']
        else:
            stats['bestPattern'] = patterns[0]['id'] if patterns else ''
            stats['worstPattern'] = patterns[-1]['id'] if patterns else ''
        
        # Pattern frequency distribution
        pattern_types = [p['type'] for p in patterns]
        stats['patternFrequency'] = {}
        for ptype in set(pattern_types):
            stats['patternFrequency'][ptype] = pattern_types.count(ptype)
        
        # Cross-validation and out-of-sample estimates
        profitable_patterns = len([p for p in patterns if p['profitability'] > 0])
        stats['crossValidationScore'] = profitable_patterns / len(patterns) if patterns else 0
        
        # Bootstrap confidence
        stats['bootstrapConfidence'] = np.mean([p['confidence'] * p['significance'] for p in patterns])
        
        # Out-of-sample estimates (conservative)
        stats['outOfSampleResults'] = {
            'winRate': stats['avgWinRate'] * 0.85,  # 15% degradation
            'avgReturn': stats['overallProfitability'] * 0.8,  # 20% degradation
            'sharpeRatio': stats['avgSharpeRatio'] * 0.75  # 25% degradation
        }
        
        # Time frame distribution (placeholder)
        stats['timeFrameDistribution'] = {
            self.params['timeFrame']['value']: len(patterns)
        }
        
        return stats
    
    def _empty_statistics(self):
        """Return empty statistics structure"""
        return {
            'totalPatterns': 0,
            'uniquePatterns': 0,
            'avgConfidence': 0,
            'avgSupport': 0,
            'avgSignificance': 0,
            'overallProfitability': 0,
            'avgWinRate': 0,
            'avgSharpeRatio': 0,
            'bestPattern': '',
            'worstPattern': '',
            'patternFrequency': {},
            'timeFrameDistribution': {},
            'crossValidationScore': 0,
            'bootstrapConfidence': 0,
            'outOfSampleResults': {
                'winRate': 0,
                'avgReturn': 0,
                'sharpeRatio': 0
            }
        }
    
    def run_analysis(self):
        """Main analysis pipeline"""
        print(f"\n=== Starting Forex Pattern Mining Analysis ===")
        print(f"Currency Pair: {self.params['currencyPair']['symbol']}")
        print(f"Time Frame: {self.params['timeFrame']['label']}")
        print(f"Window Size: {self.params['windowSize']}")
        print(f"Min Support: {self.params['minSupport']*100:.1f}%")
        print(f"Min Confidence: {self.params['minConfidence']*100:.1f}%")
        
        # Step 1: Fetch/generate data
        print(f"\nStep 1: Data Generation")
        data = self.fetch_forex_data()
        
        # Step 2: Extract patterns
        print(f"\nStep 2: Pattern Extraction")
        patterns = self.extract_patterns(data)
        
        # Step 3: Calculate statistics
        print(f"\nStep 3: Statistical Analysis")
        statistics = self.calculate_statistics(patterns)
        
        # Step 4: Compile results
        results = {
            'patterns': patterns,
            'statistics': statistics,
            'metadata': {
                'dataPointsAnalyzed': len(data),
                'patternsFound': len(patterns),
                'executionTime': 0,  # Will be updated by Kaggle
                'algorithmVersion': '1.0.0',
                'parameters': self.params
            }
        }
        
        print(f"\n=== Analysis Complete ===")
        print(f"Patterns Found: {len(patterns)}")
        print(f"Average Confidence: {statistics['avgConfidence']:.1%}")
        print(f"Overall Profitability: {statistics['overallProfitability']:.2%}")
        
        return results

print("ForexPatternMiner class defined successfully")

In [ ]:
# Example parameters - these would be replaced by actual parameters from the dashboard
EXAMPLE_PARAMS = {
    'currencyPair': {'base': 'EUR', 'quote': 'USD', 'symbol': 'EURUSD'},
    'timeFrame': {'value': '1h', 'label': '1 Hour', 'minutes': 60},
    'windowSize': 20,
    'minSupport': 0.05,
    'minConfidence': 0.7,
    'dataPoints': 5000,
    'overlapThreshold': 0.3,
    'noiseFilter': 0.1,
    'significanceLevel': 0.05,
    'bootstrapSamples': 1000,
    'crossValidationFolds': 5,
    'includeTechnicalIndicators': True,
    'rsiPeriod': 14,
    'macdFast': 12,
    'macdSlow': 26,
    'bollBandPeriod': 20
}

print("Example parameters defined")
print(f"Currency Pair: {EXAMPLE_PARAMS['currencyPair']['symbol']}")
print(f"Data Points: {EXAMPLE_PARAMS['dataPoints']:,}")

In [ ]:
# Run the pattern mining analysis
print("Initializing pattern miner with example parameters...")
miner = ForexPatternMiner(EXAMPLE_PARAMS)

# Execute the analysis
results = miner.run_analysis()

print("\n" + "="*60)
print("PATTERN MINING RESULTS")
print("="*60)

In [ ]:
# Display detailed results
patterns = results['patterns']
statistics = results['statistics']
metadata = results['metadata']

print(f"\nMETADATA:")
print(f"  Data Points Analyzed: {metadata['dataPointsAnalyzed']:,}")
print(f"  Patterns Found: {metadata['patternsFound']}")
print(f"  Algorithm Version: {metadata['algorithmVersion']}")

print(f"\nSTATISTICS:")
print(f"  Total Patterns: {statistics['totalPatterns']}")
print(f"  Unique Types: {statistics['uniquePatterns']}")
print(f"  Avg Confidence: {statistics['avgConfidence']:.1%}")
print(f"  Avg Support: {statistics['avgSupport']:.2%}")
print(f"  Overall Profitability: {statistics['overallProfitability']:.2%}")
print(f"  Cross-Validation Score: {statistics['crossValidationScore']:.1%}")

print(f"\nPATTERN DISTRIBUTION:")
for ptype, count in statistics['patternFrequency'].items():
    percentage = (count / statistics['totalPatterns']) * 100
    print(f"  {ptype.capitalize()}: {count} ({percentage:.1f}%)")

if patterns:
    print(f"\nTOP 5 PATTERNS BY CONFIDENCE:")
    sorted_patterns = sorted(patterns, key=lambda x: x['confidence'], reverse=True)
    for i, pattern in enumerate(sorted_patterns[:5]):
        print(f"  {i+1}. {pattern['id']} ({pattern['type']}) - {pattern['confidence']:.1%} confidence, {pattern['support']:.2%} support")

In [ ]:
# Visualization of results
if patterns:
    # Pattern type distribution
    pattern_types = [p['type'] for p in patterns]
    type_counts = {ptype: pattern_types.count(ptype) for ptype in set(pattern_types)}
    
    plt.figure(figsize=(12, 8))
    
    # Subplot 1: Pattern distribution
    plt.subplot(2, 2, 1)
    plt.pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
    plt.title('Pattern Type Distribution')
    
    # Subplot 2: Confidence vs Support scatter
    plt.subplot(2, 2, 2)
    confidences = [p['confidence'] for p in patterns]
    supports = [p['support'] for p in patterns]
    colors = ['red' if p['type'] == 'bearish' else 'green' if p['type'] == 'bullish' else 'blue' for p in patterns]
    plt.scatter(confidences, supports, c=colors, alpha=0.6)
    plt.xlabel('Confidence')
    plt.ylabel('Support')
    plt.title('Confidence vs Support')
    
    # Subplot 3: Profitability distribution
    plt.subplot(2, 2, 3)
    profitabilities = [p['profitability'] for p in patterns if p['profitability'] is not None]
    plt.hist(profitabilities, bins=15, alpha=0.7, color='purple')
    plt.xlabel('Profitability')
    plt.ylabel('Frequency')
    plt.title('Profitability Distribution')
    
    # Subplot 4: Sample pattern shapes
    plt.subplot(2, 2, 4)
    for i, pattern in enumerate(patterns[:3]):  # Show first 3 patterns
        if pattern['pricePoints']:
            color = 'red' if pattern['type'] == 'bearish' else 'green' if pattern['type'] == 'bullish' else 'blue'
            plt.plot(pattern['pricePoints'], label=f"{pattern['type']} ({pattern['confidence']:.1%})", color=color, alpha=0.7)
    plt.xlabel('Time Steps')
    plt.ylabel('Normalized Price')
    plt.title('Sample Pattern Shapes')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\nVisualization complete!")
else:
    print("\nNo patterns found for visualization.")

In [ ]:
# Output results in JSON format for API consumption
print("\n" + "="*50)
print("JSON RESULTS FOR API")
print("="*50)

# Convert numpy types to native Python types for JSON serialization
def convert_numpy_types(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    return obj

# Convert results
json_results = convert_numpy_types(results)

# Output JSON
json_output = json.dumps(json_results, indent=2)
print(json_output)

print(f"\nAnalysis complete! Found {len(patterns)} patterns with {statistics['avgConfidence']:.1%} average confidence.")